Install stable-baselines3 for training algorithms

In [1]:
!pip install stable_baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.6 MB/s eta 0:00:00
  Using cached libtorrent-2.0.7-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (8.6 MB)
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616823 sha256=68eb0ac5bdd00e0b3ca3c85845cbd84ab97dce0f9be7726e97abc5428983b990


Set-up code

In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
software-properties-common is already the newest version (0.99.9.10).
The following additional packages will be installed:
  libgles-dev libgles1 libglew2.1 libglvnd-dev libopengl-dev libosmesa6
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev libglew2.1
  libglvnd-dev libopengl-dev libosmesa6 libosmesa6-dev
0 upgraded, 10 newly installed, 0 to remove and 27 not upgraded.
Need to get 3,438 kB of archives.
After this operation, 17.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1-mesa-glx amd64 21.2.6-0ubuntu0.1~20.04.2 [5,536 B]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles1 amd64 1.3.2-1~ubuntu0.20.04.2 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles-dev amd64 1.3.2-1~ubun

Unzip custom-hopper environment and utils

In [3]:
!unzip env_images_grayscale.zip

Archive:  env_images_grayscale.zip
 extracting: env_images_grayscale/__init__.py  
   creating: env_images_grayscale/__pycache__/
  inflating: env_images_grayscale/__pycache__/__init__.cpython-37.pyc  
  inflating: env_images_grayscale/__pycache__/__init__.cpython-38.pyc  
  inflating: env_images_grayscale/__pycache__/custom_hopper.cpython-37.pyc  
  inflating: env_images_grayscale/__pycache__/custom_hopper.cpython-38.pyc  
  inflating: env_images_grayscale/__pycache__/mujoco_env.cpython-37.pyc  
  inflating: env_images_grayscale/__pycache__/mujoco_env.cpython-38.pyc  
   creating: env_images_grayscale/assets/
  inflating: env_images_grayscale/assets/hopper.xml  
  inflating: env_images_grayscale/custom_hopper.py  
  inflating: env_images_grayscale/mujoco_env.py  


Import libraries

In [1]:
import gym
from env_images_grayscale.custom_hopper import *
from stable_baselines3 import PPO
# from gym.wrappers.pixel_observation import PixelObservationWrapper
# from gym.wrappers import FrameStack  
# from gym.wrappers import FlattenObservation
# import torch 
# import numpy as np
# import matplotlib.pyplot as plt
# from RawImagesWrapper import *
# from gym.wrappers import FilterObservation
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
env = gym.make('CustomHopper-source-v0')
vect_env = DummyVecEnv([lambda: gym.make('CustomHopper-source-v0')])
vect_env = VecFrameStack(vect_env, 4)
check_env(env)

In [ ]:
# policy_kwargs = dict(
#     features_extractor_class=Custom_convLSTM,
#     features_extractor_kwargs=dict(features_dim=512),
# )
model = PPO("CnnPolicy", vect_env, verbose=1, tensorboard_log='./training_logs/grayscale100k')
model.learn(total_timesteps=100000)

Load model and continue learning

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# policy_kwargs = dict(
#     features_extractor_class=Custom_convLSTM,
#     features_extractor_kwargs=dict(features_dim=512),
# )
model = PPO("CnnPolicy", vect_env, verbose=1, tensorboard_log='./drive/MyDrive/training_logs/grayscale200k').load('./drive/MyDrive/trained_models/PPO_grayscale100k', env=vect_env)
model.learn(total_timesteps=100000, progress_bar=True)

Using cuda device
Logging to ./training_logs/grayscale100k/PPO_3


Output()

Save model for evaluation

In [ ]:
model.save('./drive/MyDrive/trained_models/PPO_grayscale200k')
# model.save('./trained_models/PPO_vect_env')

In [ ]:
mean_rwd, std_rwd = evaluate_policy(model=model, env=vect_env, n_eval_episodes=5, render=False)
print(mean_rwd, std_rwd)

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


362.08837789297104 113.68917305197205
